In [85]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import linear_model
from sklearn.kernel_approximation import RBFSampler

In [69]:
X = pd.read_csv('data/X.csv', index_col=0,)
Y = pd.read_csv('data/Y.csv',index_col=0, names=['Crime'])
test = pd.read_csv('./data/Z.csv', index_col=0)

In [44]:
# Going to try several different loss functions to see which is best
losses = ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']
models = {}
scores = {}
for i,loss in enumerate(losses):
    scores[loss] = {}

Wall time: 0 ns


In [48]:
%%time
# Get 10 scores for each loss function
for n in xrange(10):
    for i,loss in enumerate(losses):
        # each function will have its own model
        models[loss] = {}
        sgd = linear_model.SGDClassifier(loss=loss)
        models[loss]['clf'] = sgd.fit(X,Y['Crime'])
        models[loss]['score'] = sgd.score(X,Y['Crime'])
        # The nth score for each model is stored here
        scores[loss][n] = models[loss]['score']

Wall time: 20min 59s


In [54]:
#loop through all the scores to get an average for each function
for i,loss in enumerate(losses):
    scores[loss]['average'] = 0
    for n in xrange(10):
        scores[loss]['average'] += scores[loss][n]
    scores[loss]['average'] = scores[loss]['average']/10
    print loss,scores[loss]['average']

hinge 0.112823316239
log 0.0953894372638
modified_huber 0.0776248250382
squared_hinge 0.080478310436
perceptron 0.136811726908


#### Loss Function Accuracy
Looks like perceptron  beat out the default hinge by just two percent I am going to make the assumption that as accuracy increases so does precision so I will take the model with the highest average accuracy.

##### Update
I have to use Log function since the two higher loss functions don't support the probability predict method.

##### Update 2
The log loss function scored a 34 on Kaggle.

#### Update 3 
Log loss function with rbf moved all the way up to 5.39105!

In [97]:
rbf_feature = RBFSampler(gamma=1, random_state=1)
X_features = rbf_feature.fit_transform(X)
test_features = rbf_feature.fit_transform(test) 

In [88]:

log_clf = linear_model.SGDClassifier(loss='log')
%time log_clf.fit(X_features,Y['Crime'])
print log_clf.score(X_features,Y['Crime'])


Wall time: 1min 50s
0.199191616869


In [98]:
%time results = log_clf.predict_proba(test_features)

Wall time: 2.1 s


In [99]:
df = pd.read_csv('data/train.csv')
resultdf = pd.DataFrame(results, columns = df.Category.unique())
resultdf = resultdf[sorted(resultdf.columns)]
resultdf.insert(0, 'Id', test.index.values)
resultdf.fillna(0,inplace=True)
resultdf.to_csv('submissions/sgdrbf_prob.csv', index=False)